In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
E_B = 100
P_Battery_Max = 50
SOC_UP =  soc_upper_bound = 0.9
SOC_FLOOR = soc_floor_bound = 0.1
eta_Bc=eta_Bd=0.95

# 读取数据
data_A = pd.read_csv("data/园区A.csv")
data_B = pd.read_csv("data/园区B.csv")
data_C = pd.read_csv("data/园区C.csv")

P_user_A = data_A["用电量"]
P_user_B = data_B["用电量"]
P_user_C = data_C["用电量"]
P_PV_A = data_A["光伏供电量"]
P_WG_B = data_B["风电供电量"]
P_PV_C = data_C["光伏供电量"]
P_WG_C = data_C["风电供电量"]


In [40]:
# C
init_SOC=0.5


SOC=[init_SOC]

def setSOC(t,value):
    SOC[t]=value

def getSOC(t):
    if t<0:
        t=0
    return SOC[t]


def get_SOC_new(SOC_last,P_B):
    return SOC_last+P_B/E_B


def gamma_B(P_B):
    if P_B<0:
        return eta_Bc
    else:
        return 1/eta_Bd

def get_P_B(Delta_P):
    P_Bdmax = P_Bcmax = P_Battery_Max
    if Delta_P>-P_Bdmax and Delta_P<=P_Bcmax:
        return Delta_P
    elif Delta_P>P_Bcmax:
        return P_Bcmax
    else:
        return -P_Bdmax

# 放电时为正，充电时为负
def cal_P_B(P_Battery_Max,Delta_P,t):
    P_B=get_P_B(Delta_P) 
    print(t)
    SOC_last = getSOC(t-1) # t=11时，getSOC(10) 越界？
    SOC_new = get_SOC_new(SOC_last,Delta_P) #<=SOC_UP
    SOC_new = SOC_last-gamma_B(Delta_P)*P_B*1/E_B
    if SOC_new > SOC_UP:
        SOC_new = SOC_UP
    elif SOC_new < SOC_FLOOR:
        SOC_new = SOC_FLOOR
    
    P_B=(SOC_last-SOC_new)*E_B/gamma_B(Delta_P) 
    if(P_B>0):
        print("放电{}kW",P_B)
    else:
        print("充电{}kW",P_B)
    return P_B
    
    

# data_C['Curt_C'] = data_C.iloc[:, 1] + data_C.iloc[:, 2] - data_C.iloc[:, 0]

def find_lack(t):
    data = P_PV_A+P_PV_C-P_user_C
    for i in range(t+1,len(data)):
        if data[i] < 0:
            return i
    return -1




def wait_sun(P_wind,t):
    j = t
    n = find_lack(t)
    lack = P_user_C[n]-P_PV_C[n]-P_WG_C[n]
    extra=0
    for k in range(j+1,n):
        extra+=cal_P_B(P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k])
    B_empty = (SOC_UP-getSOC(t))*E_B
    if(lack-extra>B_empty):
        P_B = cal_P_B(P_Battery_Max,-P_wind,t) 
        P_wind+=P_B
        Battery-=P_B 
        C_wind=-P_B 
        
    while(n<=365*24):
        j = n+1
        n = find_lack(n)
        lack = P_user_C[n]-P_PV_C[n]-P_WG_C[n]
        extra=0
        for k in range(j+1,n):
            extra+=cal_P_B(P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k])
        B_empty = (SOC_UP-getSOC(t))*E_B
        if(lack-extra>B_empty):
            P_B = cal_P_B(P_Battery_Max,-P_wind,t) 
            P_wind+=P_B
            Battery-=P_B 
            C_wind=-P_B 
    return P_wind,C_wind,Battery 

        
    
# Battery = init_SOC*E_B or 0
def Process(Battery,P_user, P_light, P_wind,label,t):
    
    print("现在是{}:00",t)
    
    # 光
    C_light=0
    C_wind=0 
    C_net=0
    if P_light > P_user:
        C_light = P_user
        P_user = 0
        P_light -= C_light
        P_B = cal_P_B(P_Battery_Max,-P_light,t)
        # charge
        P_light-=P_B 
        Battery+=P_B
    else:
        C_light = P_light
        P_user -= P_light
    
    if P_user == 0:
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net
    
    # 电池
    # discharge
    P_B = cal_P_B(P_Battery_Max,P_user,t) #放电时为正，充电时为负
    Battery += P_B 
    P_user +=P_B 
    if P_user==0:
        P_B = cal_P_B(P_Battery_Max,-P_wind,t) 
        P_wind+=P_B
        Battery-=P_B 
        C_wind=-P_B 
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net
    
    # 风
    if P_wind > P_user:
        C_wind = P_user
        P_user = 0
        P_wind-=C_wind

        if label == 'B':
            P_B = cal_P_B(P_Battery_Max,-P_wind,t)
            C_wind -= P_B
            P_wind +=P_B 
            Battery-=P_B 
            
        if label == 'C':
            # 额外考虑
            P_wind,C_wind,Battery = wait_sun(P_wind,t)#################
            
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net

        
    else:
        C_wind = P_wind
        P_user -= P_wind
    
    if P_user > 0:
        C_net = P_user
        P_user = 0
    # SOC[t]=Battery/E_B
    SOC.append(Battery/E_B)
    return C_light, C_wind, C_net





In [41]:
# A
cost_A = []
for t in range(0,24):
    C_light, C_wind, C_net = Process(0,P_user_A[t], P_PV_A[t], 0,"A",t)
    print(C_light, C_wind, C_net)
    cost_A.append(C_light*0.4+C_wind*0.5+C_net*1) 
    print(cost_A[t])


sum_A = sum(cost_A)
print(cost_A)
print(sum_A)

现在是{}:00 0
0
放电{}kW 38.0
0.0 0 313.0
313.0
现在是{}:00 1
1
放电{}kW 38.0
0.0 0 313.0
313.0
现在是{}:00 2
2
放电{}kW 26.600000000000005
0.0 0 303.6
303.6
现在是{}:00 3
3
放电{}kW 26.600000000000005
0.0 0 336.6
336.6
现在是{}:00 4
4
放电{}kW 15.770000000000005
0.0 0 325.77
325.77
现在是{}:00 5
5
放电{}kW 15.770000000000005
0.0 0 308.77
308.77
现在是{}:00 6
6
放电{}kW 5.481500000000006
0.0 0 298.4815
298.4815
现在是{}:00 7
7
放电{}kW 5.481500000000006
4.35 0 381.13149999999996
382.87149999999997
现在是{}:00 8
8
充电{}kW -4.292574999999996
226.95 0 143.757425
234.537425
现在是{}:00 9
9
充电{}kW -50.0
281 0 0
112.4
现在是{}:00 10
10
充电{}kW -50.0
447 0 0
178.8
现在是{}:00 11
11
充电{}kW -63.15789473684211
447 0 0
178.8
现在是{}:00 12
12
充电{}kW -63.15789473684211
447 0 0
178.8
现在是{}:00 13
13
充电{}kW -77.00831024930748
405 0 0
162.0
现在是{}:00 14
14
充电{}kW -77.00831024930748
404 0 0
161.60000000000002
现在是{}:00 15
15
充电{}kW -82.65789473684211
318.15000000000003 0 2.192105263157856
129.45210526315788
现在是{}:00 16
16
充电{}kW -82.65789473684211
46.425 0 138

In [ ]:
E_B = 100
P_Battery_Max = 50


# 读取数据
data_A = pd.read_csv("data/园区A.csv")
data_B = pd.read_csv("data/园区B.csv")
data_C = pd.read_csv("data/园区C.csv")

df_A = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])
df_B = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])
df_C = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])


while(E_B<=1000):
    P_Battery_Max = 50
    while(P_Battery_Max<=1000):
        # 初始化变量
        Battery = 0
        init_Battery_capacity_floor_bound = E_B * soc_floor_bound
        init_Battery_capacity_upper_bound = E_B * soc_upper_bound

        cost_of_Battery=(800*P_Battery_Max+1800*E_B)/3650

        # 处理数据
        #print('E_B:',E_B,'P_Battery_Max:',P_Battery_Max)
        # A处理
        n_rows = data_A.shape[0]

        # 创建一个长度与data_A行数相同的空列向量
        A_light = np.zeros((n_rows,1))
        A_wind = np.zeros((n_rows,1))
        A_net = np.zeros((n_rows,1))

        # 循环遍历data_A的每一行
        for index, row in data_A.iterrows():
            P_user = row['用电量']
            P_light = row['光伏供电量']
            P_wind = 0
            A_light[index],A_wind[index],A_net[index]=Process(P_user, P_light, P_wind,data_A,'A',index)

        # print(A_light)
        # print(A_wind)
        # print(A_net)
        cost_A=1*A_net+0.4*A_light
        #print('A成本:',sum(cost_A)+cost_of_Battery)


        #B处理
        n_rows = data_B.shape[0]

        # 创建一个长度与data_B行数相同的空列向量
        B_light = np.zeros((n_rows,1))
        B_wind = np.zeros((n_rows,1))
        B_net = np.zeros((n_rows,1))

        # 循环遍历data_B的每一行
        for index, row in data_B.iterrows():
            P_user = row['用电量']
            P_wind = row['风电供电量']
            P_light = 0
            B_light[index],B_wind[index],B_net[index]=Process(P_user, P_light, P_wind,data_B,'B',index)

        #print(B_light)
        #print(B_wind)
        #print(B_net)
        cost_B=1*B_net+0.5*B_wind
        #print('B风电购入',sum(B_wind))
        #print('B电网购入',sum(B_net))
        #print('B成本:',sum(cost_B)+cost_of_Battery)

        # C处理
        n_rows = data_C.shape[0]

        # 创建一个长度与data_C行数相同的空列向量
        C_light = np.zeros((n_rows,1))
        C_wind = np.zeros((n_rows,1))
        C_net = np.zeros((n_rows,1))

        data_C['Curt_C'] = data_C.iloc[:, 1] + data_C.iloc[:, 2] - data_C.iloc[:, 0]
        #print(data_C['Curt_C'])
        # 循环遍历data_C的每一行
        for index, row in data_C.iterrows():
            P_user = row['用电量']
            P_light = row['光伏供电量']
            P_wind = row['风电供电量']
            C_light[index],C_wind[index],C_net[index]=Process(P_user, P_light, P_wind,data_C,'C',index)

        #print(C_light)
        #print(C_wind)
        #print(C_net)
        cost_C=1*C_net+0.4*C_light+0.5*C_wind
        cost_A = np.array(cost_A)
        cost_A= cost_A.reshape(-1)
        cost_B = np.array(cost_B)
        cost_B= cost_B.reshape(-1)
        cost_C = np.array(cost_C)
        cost_C= cost_C.reshape(-1)
        #print('C成本:',sum(cost_C)+cost_of_Battery)
        #print('----------------------------------')
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_A) + cost_of_Battery]})
        df_A = pd.concat([df_A, new_row], ignore_index=True)
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_B) + cost_of_Battery]})
        df_B = pd.concat([df_B, new_row], ignore_index=True)
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_C) + cost_of_Battery]})
        df_C = pd.concat([df_C, new_row], ignore_index=True)
        P_Battery_Max+=50
    
    E_B+=100


/tmp/ipykernel_5235/1460002269.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_A = pd.concat([df_A, new_row], ignore_index=True)
/tmp/ipykernel_5235/1460002269.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_B = pd.concat([df_B, new_row], ignore_index=True)
/tmp/ipykernel_5235/1460002269.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determin

In [ ]:
# 输出结果

print(df_A)
print(df_B)
print(df_C)

df_A.to_excel('./园区A_cost.xlsx')
df_B.to_excel('./园区B_cost.xlsx')
df_C.to_excel('./园区C_cost.xlsx')
# 画出E_B,P_Battery_Max与cost的关系
# 创建一个新的图形
# Create a 3D scatter plot
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] =False

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_A['E_B'], df_A['P_Battery_Max'], df_A['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区A E_B,P_Battery_Max与cost的关系')
plt.show()

# 保存图形到指定文件夹和文件名
# output_folder = 'problem1/pictures'
# output_filename = '园区A E_B,P_Battery_Max与cost的关系.png'
# output_path = f'{output_folder}/{output_filename}'
# plt.savefig(output_path, dpi=300)
# plt.close()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_B['E_B'], df_B['P_Battery_Max'], df_B['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区B E_B,P_Battery_Max与cost的关系')
plt.show()

# # 保存图形到指定文件夹和文件名
# output_folder = 'problem1/pictures'
# output_filename = '园区B E_B,P_Battery_Max与cost的关系.png'
# output_path = f'{output_folder}/{output_filename}'
# plt.savefig(output_path, dpi=300)
# plt.close()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_C['E_B'], df_C['P_Battery_Max'], df_C['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区C E_B,P_Battery_Max与cost的关系')
plt.show()

# # 保存图形到指定文件夹和文件名
# output_folder = 'problem1/pictures'
# output_filename = '园区C E_B,P_Battery_Max与cost的关系.png'
# output_path = f'{output_folder}/{output_filename}'
# plt.savefig(output_path, dpi=300)
# plt.close()

In [ ]:
# def charge(d_P):
#     global Battery
#     tmp=Battery
#     if P_Battery_Max >= d_P:
#         if Battery + d_P >= init_Battery_capacity_upper_bound:
#             Battery = init_Battery_capacity_upper_bound
#         else:
#             Battery += d_P
#     else:
#         if Battery + P_Battery_Max >= init_Battery_capacity_upper_bound:
#             Battery = init_Battery_capacity_upper_bound
#         else:
#             Battery += P_Battery_Max
#     #print('Battery充电:','delt_p:',d_P,' ',tmp,'->',Battery)
# def Cal_charge(P_wind,data,h):
#     index_lack=find_lack(data['Curt_C'],h)
#     if index_lack == -1:
#         print(f'从{h}电后面没有负荷缺口')
#         return 0

#     extra=sum(
#         data['Curt_C'][h:index_lack].apply(lambda x: P_Battery_Max if x > P_Battery_Max else x)
#         )
    
#     if extra>(SOC_UP-getSOC(t))*E_B:
        


#     lack=-data['Curt_C'][index_lack]

#     while lack<extra and index_lack<len(data) and data['Curt_C'][index_lack]<=0:
#         index_lack+=1
#         lack+=(-data['Curt_C'][index_lack])

#     if lack > extra:
#         Bu_Wind=lack-extra
#         if Bu_Wind>P_wind:
#             return P_wind
#         else:
#             return Bu_Wind
#     else:
#         return 0 